<a href="https://colab.research.google.com/github/AtheerAlzhrani/NLP/blob/main/Classification_of_the_sarcasm_headlines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size=10000
embedding_dim=16
max_length=100
trunc_type='post'
padding_type='post'
oov_tok="<OOV>"
training_size=20000

In [ ]:
!wget --no-check-certificate \
     https://storage.googleapis.com/learning-datasets/sarcasm.json  \
     -O /tmp/sarcasm.json

--2024-04-14 09:03:54--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.182.207, 64.233.183.207, 173.194.193.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.182.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2024-04-14 09:03:54 (189 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
with open("/tmp/sarcasm.json",'r') as f:
  datastore=json.load(f)
sentences=[]
labels=[]

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [ ]:
training_sentences=sentences[0:training_size]
testing_sentences=sentences[training_size:]

training_labels=labels[0:training_size]
testing_labels=labels[training_size:]

In [ ]:
from os import truncate
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index=tokenizer.word_index

training_sequences=tokenizer.texts_to_sequences(training_sentences)
training_padded=pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [ ]:
import numpy as np

training_padded=np.array(training_padded)
training_labels=np.array(training_labels)
testing_padded=np.array(testing_padded)
testing_labels=np.array(testing_labels)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_epochs=30
history=model.fit

Epoch 1/30
625/625 [==============================] - 4s 5ms/step - loss: 0.6604 - accuracy: 0.5900
Epoch 2/30
625/625 [==============================] - 3s 5ms/step - loss: 0.4153 - accuracy: 0.8395
Epoch 3/30
625/625 [==============================] - 3s 4ms/step - loss: 0.3013 - accuracy: 0.8823
Epoch 4/30
625/625 [==============================] - 3s 5ms/step - loss: 0.2509 - accuracy: 0.9003
Epoch 5/30
625/625 [==============================] - 3s 5ms/step - loss: 0.2161 - accuracy: 0.9165
Epoch 6/30
625/625 [==============================] - 3s 5ms/step - loss: 0.1890 - accuracy: 0.9291
Epoch 7/30
625/625 [==============================] - 3s 6ms/step - loss: 0.1674 - accuracy: 0.9387
Epoch 8/30
625/625 [==============================] - 5s 8ms/step - loss: 0.1501 - accuracy: 0.9455
Epoch 9/30
625/625 [==============================] - 3s 5ms/step - loss: 0.1361 - accuracy: 0.9507
Epoch 10/30
625/625 [==============================] - 4s 6ms/step - loss: 0.1220 - accuracy: 0.9568

In [ ]:
e=model.layers[0]
weights=e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
sentences=['robin williams inflicted on holiday moviegoers for eighth straight year','what it\'s like to lose everything in a flood']
sequences=tokenizer.texts_to_sequences(sentences)
padded=pad_sequences(sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 42ms/step
[[9.9830168e-01]
 [6.4055534e-06]]
